In [3]:
from sqlalchemy import create_engine
import pandas as pd

connection_string = (
    "mssql+pyodbc://@KCLNK\\SQLEXPRESS/TransactionsDB"
    "?driver=ODBC+Driver+17+for+SQL+Server"
    "&trusted_connection=yes"
)

engine = create_engine(connection_string)

In [ ]:


users_data_df = pd.read_csv("Files/users_data.csv")
cards_data_df = pd.read_csv("Files/cards_data.csv")

money_cols = ['per_capita_income', 'yearly_income', 'total_debt']

for col in money_cols:
    users_data_df[col] = (
        users_data_df[col]
        .replace(r'[\$,]', '', regex=True)   # Remove $
        .astype(float)                     # Convert to float
    )
    
cards_data_df["credit_limit"] = cards_data_df["credit_limit"].replace(r'[\$,]', '', regex=True).astype(float)

# importing files into SQLserver
users_data_df.to_sql("Users", engine, if_exists="append", index=False)


# Done, IGNORE THE ERROR


IntegrityError: (pyodbc.IntegrityError) ('23000', "[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Violation of PRIMARY KEY constraint 'PK__Users__3213E83FCFA7E1F3'. Cannot insert duplicate key in object 'dbo.Users'. The duplicate key value is (825). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621)")
[SQL: INSERT INTO [Users] (id, current_age, retirement_age, birth_year, birth_month, gender, address, latitude, longitude, per_capita_income, yearly_income, total_debt, credit_score, num_credit_cards) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?,  ... 6406 characters truncated ...  ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: (825, 53, 66, 1966, 11, 'Female', '462 Rose Lane', 34.15, -117.76, 29278.0, 59696.0, 127613.0, 787, 5, 1746, 53, 68, 1966, 12, 'Female', '3606 Federal Boulevard', 40.76, -73.74, 37891.0, 77254.0, 19

In [ ]:
cards_data_df['expires'] = pd.to_datetime(cards_data_df['expires'], format='%m/%Y', errors='coerce')
cards_data_df['acct_open_date'] = pd.to_datetime(cards_data_df['acct_open_date'], format='%m/%Y', errors='coerce')
cards_data_df["has_chip"] = cards_data_df["has_chip"].map({'YES': True, 'NO': False})

cards_data_df.to_sql("Cards", engine, if_exists="append", index=False)


28

In [38]:
cards_data_df.head()

card_id  client_id  card_brand        card_type       card_number  \
0     4524        825        Visa            Debit  4344676511950444   
1     2731        825        Visa            Debit  4956965974959986   
2     3701        825        Visa            Debit  4582313478255491   
3       42        825        Visa           Credit  4879494103069057   
4     4659        825  Mastercard  Debit (Prepaid)  5722874738736011   

     expires  cvv  has_chip  num_cards_issued  credit_limit acct_open_date  \
0 2022-12-01  623      True                 2       24295.0     2002-09-01   
1 2020-12-01  393      True                 2       21968.0     2014-04-01   
2 2024-02-01  719      True                 2       46414.0     2003-07-01   
3 2024-08-01  693     False                 1       12400.0     2003-01-01   
4 2009-03-01   75      True                 1          28.0     2008-09-01   

   year_pin_last_changed card_on_dark_web  
0                   2008              NaN  
1                   2014              NaN  
2                   2004              NaN  
3                   2012              NaN  
4                   2009              NaN

In [40]:
transactions_data_df = pd.read_csv("Files/transactions_data.csv")

In [41]:
transactions_data_df["amount"] = transactions_data_df["amount"].replace(r'[\$,]', '', regex=True).astype(float)

In [46]:
transactions_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13305915 entries, 0 to 13305914
Data columns (total 12 columns):
 #   Column          Dtype  
---  ------          -----  
 0   id              int64  
 1   date            object 
 2   client_id       int64  
 3   card_id         int64  
 4   amount          float64
 5   use_chip        object 
 6   merchant_id     int64  
 7   merchant_city   object 
 8   merchant_state  object 
 9   zip             float64
 10  mcc             int64  
 11  errors          object 
dtypes: float64(2), int64(5), object(5)
memory usage: 1.2+ GB


In [47]:
transactions_data_df.rename(columns={'id': 'transaction_id'}, inplace = True)
transactions_data_df.rename(columns={'date': 'transaction_date'}, inplace = True)
transactions_data_df.rename(columns={'client_id': 'id'}, inplace = True)

In [48]:
transactions_data_df['transaction_date'] = pd.to_datetime(transactions_data_df['transaction_date'], errors='coerce')

In [49]:
transactions_data_df['zip'] = transactions_data_df['zip'].astype('object')

In [50]:
transactions_data_df.to_sql("Transactions", engine, if_exists="append", index= False)

135

In [4]:
import json

with open("Files/train_fraud_labels.json", "r", encoding="utf-8") as fraud_file:
    fraud_data = json.load(fraud_file)
    
fraud_dictionary = fraud_data["target"]
fraud_df = pd.DataFrame(list(fraud_dictionary.items()), columns=["transaction_id", "fraud_label"])

fraud_df.head(10)

transaction_id fraud_label
0       10649266          No
1       23410063          No
2        9316588          No
3       12478022          No
4        9558530          No
5       12532830          No
6       19526714          No
7        9906964          No
8       13224888          No
9       13749094          No

In [6]:
fraud_df.dtypes

transaction_id    object
fraud_label       object
dtype: object

In [7]:
fraud_df["fraud_label"] = fraud_df["fraud_label"].map({'Yes': True, 'No': False})
fraud_df["transaction_id"] = fraud_df["transaction_id"].astype("Int64")

In [10]:
fraud_df.dtypes # check
fraud_df.head(10)

transaction_id  fraud_label
0        10649266        False
1        23410063        False
2         9316588        False
3        12478022        False
4         9558530        False
5        12532830        False
6        19526714        False
7         9906964        False
8        13224888        False
9        13749094        False

In [11]:
fraud_df.to_sql("Fraud_Labels", engine, if_exists="append", index=False)

963

In [13]:
print(len(fraud_dictionary), 'records') # checks with the dataframe imported to SQL Database '8914963 records'

8914963 records


In [14]:
with open("Files/mcc_codes.json") as mcc_codes_file:
    mcc_map = json.load(mcc_codes_file)

mcc_df = pd.DataFrame(list(mcc_map.items()), columns=['mcc_code', 'merchant_name'])

In [15]:
mcc_df.head(10)

mcc_code                               merchant_name
0     5812               Eating Places and Restaurants
1     5541                            Service Stations
2     7996        Amusement Parks, Carnivals, Circuses
3     5411                Grocery Stores, Supermarkets
4     4784                       Tolls and Bridge Fees
5     4900  Utilities - Electric, Gas, Water, Sanitary
6     5942                                 Book Stores
7     5814                       Fast Food Restaurants
8     4829                              Money Transfer
9     5311                           Department Stores

In [16]:
mcc_df.dtypes

mcc_code         object
merchant_name    object
dtype: object

In [17]:
mcc_df["mcc_code"] = mcc_df["mcc_code"].astype("int") 

In [18]:
mcc_df.dtypes

mcc_code          int64
merchant_name    object
dtype: object

In [19]:
mcc_df.count()

mcc_code         109
merchant_name    109
dtype: int64

In [25]:
mcc_df.rename(columns={"mcc_code":"mcc", "merchant_name":"mcc_category"}, inplace=True)

In [26]:
mcc_df.to_sql("MCC_Codes", engine, if_exists="append", index=False)

109